In [42]:
!pip install pythainlp nemo_toolkit['all']==1.0.0rc1

  Using cached nemo_toolkit-1.0.0rc1-py3-none-any.whl (737 kB)
  Using cached torchaudio-0.7.2-cp38-cp38-manylinux1_x86_64.whl (7.6 MB)
  Using cached pytorch_lightning-1.1.5-py3-none-any.whl (685 kB)
  Using cached torchtext-0.8.0-cp38-cp38-manylinux1_x86_64.whl (7.0 MB)
  Using cached torch-1.7.1-cp38-cp38-manylinux1_x86_64.whl (776.8 MB)
  Using cached torchvision-0.8.2-cp38-cp38-manylinux1_x86_64.whl (12.8 MB)
  Using cached frozendict-2.3.8-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (113 kB)
  Using cached pystoi-0.3.3.tar.gz (7.0 kB)
  Preparing metadata (setup.py) ... done
  Using cached spectralcluster-0.2.18-py3-none-any.whl (28 kB)
  Using cached isort-4.3.21-py2.py3-none-any.whl (42 kB)
  Using cached pesq-0.0.4.tar.gz (38 kB)
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement megatron-lm==1.1.5; extra == "all" (from nemo-toolkit[all]) (from versions: 2.2.1, 2.2.2, 2.2.3, 2.2.4)
ERROR: No matching distributio

In [1]:
import librosa
import concurrent.futures
from tqdm.auto import tqdm
import os
from scipy.io import wavfile
import warnings

warnings.filterwarnings("ignore")

i=0
def change_sr(wav):
    filepath = os.path.join('clean_voice', wav)
    y, sr = librosa.load(filepath, sr=22050) # Downsample 44.1kHz to 22050kHz
    out_file_path = os.path.join("c-22050", wav)
    wavfile.write(out_file_path, sr,data=y)

with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:
    future_to_text = {executor.submit(change_sr,filename): filename for filename in os.listdir('clean_voice')} 
    for future in tqdm((concurrent.futures.as_completed(future_to_text))):
            i +=1


# for filename in tqdm(os.listdir('sel_mod')):
#     change_sr(filename)

In [2]:
for filename in os.listdir('c-22050'):
    filepath = os.path.join('c-22050', filename)
    y, sr = librosa.load(filepath, sr=22050) # Downsample 44.1kHz to 22050kHz
    print(sr)
    break

22050


In [1]:
import os
import glob

import librosa
import numpy as np
import pandas as pd
import concurrent.futures
import warnings
from threading import Lock
import multiprocessing as mp
warnings.filterwarnings("ignore")

from pythainlp.tokenize import word_tokenize
from pythainlp.util import normalize, isthaichar

from sklearn.model_selection import train_test_split
from scipy.io import wavfile
import tqdm


def progress_indicator(future):
    global lock, tasks_total, tasks_completed
    # obtain the lock
    with lock:
        # update the counter
        tasks_completed += 1
        # report progress
        if tasks_completed % 500 == 0:
            print(f'{tasks_completed}/{tasks_total} completed, {tasks_total-tasks_completed} remain.')
            
lock = Lock()
# total tasks we will execute
tasks_total = 38207
# total completed tasks
tasks_completed = 0

global df_glo 
df_glo = pd.read_csv("transcribe_trans_clean_limit_CER.csv")

def clean_text(text):
    norm_text = normalize(text)
    norm_text = ''.join(char for char in norm_text if isthaichar(char))
    words = word_tokenize(norm_text, engine='newmm')
    words = [words[i-1] if words[i] == 'ๆ' else words[i] for i in range(len(words))]
    return ' '.join(words)

def __process(filename,inp_dir):
    try:
        text = df_glo.loc[df_glo['path'] == filename, 'sentence'].iloc[0]
        text = ' '.join(text)
        text = clean_text(text)
        
        out_file_path = os.path.join("trim_voice5", filename.replace('.wav', '_trim.wav'))
        filepath = os.path.join(inp_dir, filename)
        
        y, sr = librosa.load(filepath)
        y_trim, _ = librosa.effects.trim(y, top_db=20)
        duration = librosa.get_duration(y = y_trim, sr = sr)
        
        wavfile.write(out_file_path, sr, y_trim)
        return '{"audio_filepath": "'+out_file_path+'", \
                           "text": "'+text.strip()+'", "duration": '+str(duration)+'}\n'
    except Exception as e:
        pass
        print('error:',e)

def create_menifest(inp_dir, out_dir):
    entries = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:
        future_to_text = {executor.submit(__process,filename,inp_dir): filename for filename in os.listdir(inp_dir)} 
        for future in (concurrent.futures.as_completed(future_to_text)):
            entries.append(future.result())
            future.add_done_callback(progress_indicator)
            
    train_files, test_files = train_test_split(entries, test_size=0.05, random_state=42)

    print("Number of sentence in train set : ", len(train_files))
    print("Number of sentence in test set : ", len(test_files))
    
    with open(os.path.join(out_dir, 'common_train.json'), 'w') as tr:
        for train_file in train_files:
            
            if train_file!='' and train_file is not None:
                tr.write(train_file)
    
    with open(os.path.join(out_dir, 'common_test.json'), 'w') as tt:
        for test_file in test_files:
            
            if test_file!='' and test_file is not None:
                tt.write(test_file)

create_menifest("c-22050", "trim_voice5")

500/38207 completed, 37707 remain.
1000/38207 completed, 37207 remain.
1500/38207 completed, 36707 remain.
2000/38207 completed, 36207 remain.
2500/38207 completed, 35707 remain.
3000/38207 completed, 35207 remain.
3500/38207 completed, 34707 remain.
4000/38207 completed, 34207 remain.
4500/38207 completed, 33707 remain.
5000/38207 completed, 33207 remain.
5500/38207 completed, 32707 remain.
6000/38207 completed, 32207 remain.
6500/38207 completed, 31707 remain.
7000/38207 completed, 31207 remain.
7500/38207 completed, 30707 remain.
8000/38207 completed, 30207 remain.
8500/38207 completed, 29707 remain.
9000/38207 completed, 29207 remain.
9500/38207 completed, 28707 remain.
10000/38207 completed, 28207 remain.
10500/38207 completed, 27707 remain.
11000/38207 completed, 27207 remain.
11500/38207 completed, 26707 remain.
12000/38207 completed, 26207 remain.
12500/38207 completed, 25707 remain.
13000/38207 completed, 25207 remain.
13500/38207 completed, 24707 remain.
14000/38207 completed

In [43]:
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install pytorch-lightning==1.4.4
BRANCH = 'v1.0.2'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[tts]


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-7ubuntu0.1).
libsndfile1 set to manually installed.
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 sox
0 upgraded, 6 newly installed, 0 to remove and 100 not upgraded.
Need to get 513 kB of archives.
After this operation, 1564 kB of additional disk space will be used.
E: You don't have enough free space in /var/cache/apt/archives/.
  Using cached Unidecode-1.3.6-py3-none-any.whl (235 kB)

[notice] A new release of p

In [44]:
!wget https://raw.githubusercontent.com/NVIDIA/NeMo/v1.0.2/examples/tts/tacotron2.py
!mkdir conf && cd conf && wget https://raw.githubusercontent.com/NVIDIA/NeMo/v1.0.2/examples/tts/conf/tacotron2.yaml && cd ..

--2023-06-13 02:52:37--  https://raw.githubusercontent.com/NVIDIA/NeMo/v1.0.2/examples/tts/tacotron2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1874 (1.8K) [text/plain]
Saving to: ‘tacotron2.py’

tacotron2.py        100%[===================>]   1.83K  --.-KB/s    in 0s      

2023-06-13 02:52:37 (10.5 MB/s) - ‘tacotron2.py’ saved [1874/1874]

--2023-06-13 02:52:38--  https://raw.githubusercontent.com/NVIDIA/NeMo/v1.0.2/examples/tts/conf/tacotron2.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3706 (3.6K) [text/plain]

In [45]:
!wget "https://drive.google.com/uc?id=1Kb78OlATEHuGHly1zGWgmFnckc-A-xgy&export=download" -O "tacotron2_th.yaml"

--2023-06-13 02:55:32--  https://drive.google.com/uc?id=1Kb78OlATEHuGHly1zGWgmFnckc-A-xgy&export=download
Resolving drive.google.com (drive.google.com)... 172.217.26.78, 2404:6800:4001:810::200e
Connecting to drive.google.com (drive.google.com)|172.217.26.78|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-8k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/918dh0um2ui33ot1i7s7rfej2tei2rki/1686650100000/07480652331059926743/*/1Kb78OlATEHuGHly1zGWgmFnckc-A-xgy?e=download&uuid=1a55856b-a08a-4ad9-b477-c1fbbc9e0abb [following]
--2023-06-13 02:55:36--  https://doc-08-8k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/918dh0um2ui33ot1i7s7rfej2tei2rki/1686650100000/07480652331059926743/*/1Kb78OlATEHuGHly1zGWgmFnckc-A-xgy?e=download&uuid=1a55856b-a08a-4ad9-b477-c1fbbc9e0abb
Resolving doc-08-8k-docs.googleusercontent.com (doc-08-8k-docs.googleusercontent.com)... 216.58.196.33, 2404:6800:4001:807:

In [52]:
!python --version
!pip --version
!pip show torch
!pip show pytorch_lightning

Python 3.8.10
pip 23.0.1 from /usr/local/lib/python3.8/dist-packages/pip (python 3.8)
Name: torch
Version: 1.13.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.8/dist-packages
Requires: nvidia-cublas-cu11, nvidia-cuda-nvrtc-cu11, nvidia-cuda-runtime-cu11, nvidia-cudnn-cu11, typing-extensions
Required-by: deepspeed, kornia, megatron-core, nemo-toolkit, pytorch-lightning, sentence-transformers, torchmetrics, torchvision, triton
Name: pytorch-lightning
Version: 1.4.4
Summary: PyTorch Lightning is the lightweight PyTorch wrapper for ML researchers. Scale your models. Write less boilerplate.
Home-page: https://github.com/PyTorchLightning/pytorch-lightning
Author: William Falcon et al.
Author-email: waf2107@columbia.edu
License: Apache-2.0
Location: /usr/local/lib/python3.8/dist-packages
Requires: fsspec, future, numpy, 

In [65]:
!pip install pytorch-lightning==1.2.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.9/841.9 kB 20.3 MB/s eta 0:00:0000:01
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.9.0
    Uninstalling pytorch-lightning-1.9.0:
      Successfully uninstalled pytorch-lightning-1.9.0

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import pytorch_lightning as pl

from nemo.collections.common.callbacks import LogEpochTimeCallback
from nemo.collections.tts.models import Tacotron2Model, WaveGlowModel
from omegaconf import OmegaConf, DictConfig

from nemo.utils.exp_manager import exp_manager

[NeMo W 2023-06-28 23:11:08 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [ ]:
import os
from nemo.core.optim.lr_scheduler import CosineAnnealing

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

cfg = OmegaConf.load("tacotron2_th2-Copy1.yaml")

#print(cfg)
# sched = {'name': 'CosineAnnealing', 'warmup_steps': 1000, 'min_lr': 1e-6}
# sched = OmegaConf.create(sched)  # Convert it into a DictConfig

# Assign it to cfg.optim.sched namespace
#cfg.model.optim.sched = sched
trainer = pl.Trainer(**cfg.trainer)
#print('1')
exp_manager(trainer, cfg.get("exp_manager", None))

model = Tacotron2Model(cfg=cfg.model, trainer=trainer)
#model.setup_optimization(optim_config=cfg.model.optim)

# Let's add a few more callbacks
lr_logger = pl.callbacks.LearningRateMonitor()
epoch_time_logger = LogEpochTimeCallback()
trainer.callbacks.extend([lr_logger, epoch_time_logger])
# Call lightning trainer's fit() to train the model
#print('2')
trainer.fit(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]


[NeMo I 2023-06-28 23:12:20 exp_manager:208] Experiments will be logged at /workspace/nemo_experiments/Tacotron2_clean3/2023-06-28_23-12-20
[NeMo I 2023-06-28 23:12:20 exp_manager:548] TensorboardLogger has been set up
[NeMo I 2023-06-28 23:12:21 collections:173] Dataset loaded with 36296 files totalling 20.89 hours
[NeMo I 2023-06-28 23:12:21 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-06-28 23:12:22 collections:173] Dataset loaded with 1911 files totalling 1.09 hours
[NeMo I 2023-06-28 23:12:22 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-06-28 23:12:22 features:240] PADDING: 16
[NeMo I 2023-06-28 23:12:22 features:256] STFT using torch


initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1


[NeMo I 2023-06-28 23:12:28 modelPT:685] Optimizer config = Adam (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        eps: 1e-08
        lr: 0.001
        weight_decay: 1e-06
    )
[NeMo I 2023-06-28 23:12:28 lr_scheduler:617] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f1fb04bf670>" 
    will be used during training (effective maximum steps = 2270000) - 
    Parameters : 
    (min_lr: 1.0e-06
    max_steps: 2270000
    )



  | Name                       | Type               | Params
------------------------------------------------------------------
0 | audio_to_melspec_precessor | FilterbankFeatures | 0     
1 | text_embedding             | Embedding          | 35.3 K
2 | encoder                    | Encoder            | 5.5 M 
3 | decoder                    | Decoder            | 18.3 M
4 | postnet                    | Postnet            | 4.3 M 
5 | loss                       | Tacotron2Loss      | 0     
------------------------------------------------------------------
28.2 M    Trainable params
0         Non-trainable params
28.2 M    Total params


[NeMo W 2023-06-28 23:12:30 patch_utils:49] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.
[NeMo W 2023-06-28 23:12:33 tacotron2:366] Reached max decoder steps 1000.
[NeMo W 2023-06-28 23:12:35 tacotron2:366] Reached max decoder steps 1000.


[NeMo W 2023-06-29 03:12:58 tacotron2:366] Reached max decoder steps 1000.
[NeMo W 2023-06-29 03:13:01 tacotron2:366] Reached max decoder steps 1000.
[NeMo W 2023-06-29 03:13:04 tacotron2:366] Reached max decoder steps 1000.
[NeMo W 2023-06-29 03:13:07 tacotron2:366] Reached max decoder steps 1000.
[NeMo W 2023-06-29 03:13:11 tacotron2:366] Reached max decoder steps 1000.
[NeMo W 2023-06-29 03:13:14 tacotron2:366] Reached max decoder steps 1000.
[NeMo W 2023-06-29 03:13:17 tacotron2:366] Reached max decoder steps 1000.
[NeMo W 2023-06-29 03:13:20 tacotron2:366] Reached max decoder steps 1000.
[NeMo W 2023-06-29 03:13:22 tacotron2:366] Reached max decoder steps 1000.
[NeMo W 2023-06-29 03:13:25 tacotron2:366] Reached max decoder steps 1000.
[NeMo W 2023-06-29 03:13:29 tacotron2:366] Reached max decoder steps 1000.
[NeMo W 2023-06-29 03:13:34 tacotron2:366] Reached max decoder steps 1000.
[NeMo W 2023-06-29 03:13:38 tacotron2:366] Reached max decoder steps 1000.
[NeMo W 2023-06-29 03:13:

In [3]:
model._cfg

{'labels': ['ก', 'ข', 'ค', 'ฆ', 'ง', 'จ', 'ฉ', 'ช', 'ซ', 'ฌ', 'ญ', 'ฎ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ต', 'ถ', 'ท', 'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ฤ', 'ล', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ', 'ะ', 'ั', 'า', 'ำ', 'ิ', 'ี', 'ึ', 'ื', 'ุ', 'ู', 'เ', 'แ', 'โ', 'ใ', 'ไ', 'ๅ', '็', '่', '้', '๊', '๋', '์', '#', ' '], 'train_ds': {'dataset': {'_target_': 'nemo.collections.asr.data.audio_to_text.AudioToCharDataset', 'manifest_filepath': 'trim_voice/common_train.json', 'max_duration': None, 'min_duration': 0.1, 'trim': False, 'int_values': False, 'normalize': False, 'sample_rate': 22050, 'parser': 'base'}, 'dataloader_params': {'drop_last': False, 'shuffle': True, 'batch_size': 32, 'num_workers': 16}}, 'validation_ds': {'dataset': {'_target_': 'nemo.collections.asr.data.audio_to_text.AudioToCharDataset', 'manifest_filepath': 'trim_voice/common_test.json', 'max_duration': None, 'min_duration': 0.1, 'int_values': False, 'normalize': False, 'sample_rate': 22050, 